# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import cv2
import math

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.datasets import cifar10

Using TensorFlow backend.


## Load the Data

Start by importing the data from the pickle file.

In [4]:
# TODO: Implement load the data here.
#with open('train.p', 'rb') as f:
    #data = pickle.load(f)

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [3]:
# TODO: Use `train_test_split` here.
#for key,value in data.items():
    #print(key)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
#X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.15, random_state=432422)

170385408/170498071 [============================>.] - ETA: 0sUntaring file...


In [79]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [80]:
# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

In [81]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [82]:
# Initialize variables
ip_shape = len(X_train[0])*len(X_train[0])*3
num_classes = len(np.unique(y_train))
num_neurons = 128

In [83]:
# TODO: Build a two-layer feedforward neural network with Keras here.

model = Sequential()
model.add(Dense(num_neurons, input_shape=(ip_shape,)))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_23 (Dense)                 (None, 128)           393344      dense_input_12[0][0]             
____________________________________________________________________________________________________
activation_20 (Activation)       (None, 128)           0           dense_23[0][0]                   
____________________________________________________________________________________________________
dense_24 (Dense)                 (None, 43)            5547        activation_20[0][0]              
____________________________________________________________________________________________________
activation_21 (Activation)       (None, 43)            0           dense_24[0][0]                   
Total params: 398891
______________________________________________________________________

In [84]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [85]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_23 (None, 3072) (None, 128) <function linear at 0x7f5171e48510>
activation_20 (None, 128) (None, 128) <function relu at 0x7f5171e482f0>
dense_24 (None, 128) (None, 43) <function linear at 0x7f5171e48510>
activation_21 (None, 43) (None, 43) <function softmax at 0x7f5171e480d0>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [86]:
# TODO: Compile and train the model here.
y_train = y_train.astype(int)
y_val = y_val.astype(int)

y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(X_train.shape[0], ip_shape)
X_val = X_val.reshape(X_val.shape[0], ip_shape)

In [89]:
nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 33327 samples, validate on 5882 samples
Epoch 1/2
33327/33327 [==============================] - 5s - loss: 0.2939 - acc: 0.9131 - val_loss: 0.2997 - val_acc: 0.9114
Epoch 2/2
33327/33327 [==============================] - 5s - loss: 0.2184 - acc: 0.9352 - val_loss: 0.2724 - val_acc: 0.9230


In [90]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9230

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [83]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.15, random_state=432422)

# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

print(X_train.shape[1:])
layer1_depth = 32
filter_size = 3
num_classes = len(np.unique(y_train))
num_neurons = 128

model = Sequential()
model.add(Convolution2D(layer1_depth, filter_size, filter_size, border_mode='valid', input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_neurons))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

(32, 32, 3)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_16 (Convolution2D) (None, 30, 30, 32)    896         convolution2d_input_16[0][0]     
____________________________________________________________________________________________________
activation_23 (Activation)       (None, 30, 30, 32)    0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
flatten_10 (Flatten)             (None, 28800)         0           activation_23[0][0]              
____________________________________________________________________________________________________
dense_17 (Dense)                 (None, 128)           3686528     flatten_10[0][0]                 
_______________________________________________________________________________

In [84]:
# TODO: Compile and train the model here.
y_train = y_train.astype(int)
y_val = y_val.astype(int)


y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_val = X_val.reshape(X_val.shape[0],32, 32, 3)

nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 33327 samples, validate on 5882 samples
Epoch 1/2
33327/33327 [==============================] - 18s - loss: 0.5458 - acc: 0.8532 - val_loss: 0.1840 - val_acc: 0.9466
Epoch 2/2
33327/33327 [==============================] - 17s - loss: 0.1055 - acc: 0.9709 - val_loss: 0.1417 - val_acc: 0.9582


In [85]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9582

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [86]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.15, random_state=432422)

# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

layer1_depth = 32
filter_size = 5
num_classes = len(np.unique(y_train))
num_neurons = 128

model = Sequential()
model.add(Convolution2D(layer1_depth, filter_size, filter_size, border_mode='valid', input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(num_neurons))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_17 (Convolution2D) (None, 32, 32, 32)    2432        convolution2d_input_17[0][0]     
____________________________________________________________________________________________________
activation_26 (Activation)       (None, 32, 32, 32)    0           convolution2d_17[0][0]           
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 16, 16, 32)    0           activation_26[0][0]              
____________________________________________________________________________________________________
flatten_11 (Flatten)             (None, 8192)          0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

In [87]:
# TODO: Compile and train the model here.
y_train = y_train.astype(int)
y_val = y_val.astype(int)


y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_val = X_val.reshape(X_val.shape[0], 32, 32, 3)

nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 33327 samples, validate on 5882 samples
Epoch 1/2
33327/33327 [==============================] - 11s - loss: 0.5907 - acc: 0.8396 - val_loss: 0.1879 - val_acc: 0.9483
Epoch 2/2
33327/33327 [==============================] - 10s - loss: 0.1240 - acc: 0.9641 - val_loss: 0.1490 - val_acc: 0.9583


In [88]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9583

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [4]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
#X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.15, random_state=432422)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=432422)

# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

layer1_depth = 32
filter_size = 5
num_classes = len(np.unique(y_train))
num_neurons = 128

model = Sequential()
model.add(Convolution2D(layer1_depth, filter_size, filter_size, border_mode='valid', input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_neurons))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    2432        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 32, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 16, 16, 32)    0           activation_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 16, 16, 32)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [5]:
# TODO: Compile and train the model here.
y_train = y_train.astype(int)
y_val = y_val.astype(int)


y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_val = X_val.reshape(X_val.shape[0], 32, 32, 3)

nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 42500 samples, validate on 7500 samples
Epoch 1/2
42500/42500 [==============================] - 15s - loss: 1.4102 - acc: 0.4965 - val_loss: 1.1513 - val_acc: 0.5951
Epoch 2/2
42500/42500 [==============================] - 14s - loss: 1.1140 - acc: 0.6096 - val_loss: 1.0507 - val_acc: 0.6361


In [92]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9582

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [6]:
# TODO: Load test data
'''
with open('./test.p', mode='rb') as f:
    test_data = pickle.load(f)
X_test = test_data['features']
y_test = test_data['labels']
'''
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# TODO: Preprocess data & one-hot encode the labels
# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_test = np.array([normalize_img(image) for image in X_test], dtype=np.float32)
y_test = y_test.astype(int)
y_test = np_utils.to_categorical(y_test, nb_classes=num_classes)

X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

# TODO: Evaluate model on test data
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s     


[1.0398165121078491, 0.63929999999999998]

**Test Accuracy:** 0.8692

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.